In [69]:
! pip install datasets --quiet
! pip install evaluate --quiet
! pip install transformers --quiet
!pip install huggingface_hub --quiet

!pip install accelerate -U --quiet
!pip install transformers[torch] --quiet
!pip install shap --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 9.1 MB/s eta 0:00:00


### Generate Labels for Data

In [67]:
# create metadata (labels) to create the dataset object

import os
import csv

directory = "/content/drive/MyDrive/deceptive-16khz/"
data = []

for filename in os.listdir(directory):
    if filename.endswith(".wav"):
        label = filename.split("_")[1]
        label = 1 if label == 'lie' else 0
        data.append((filename, label))

csv_file_path = "/content/drive/MyDrive/deceptive-16khz/metadata.csv"


with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["file_name", "label"])
    writer.writerows(data)

print(f"Metadata file created at {csv_file_path}")


Metadata file created at /content/drive/MyDrive/deceptive-16khz/metadata.csv


### Split data into sliding windows

In [72]:
# Windowing function
def window_audio(audio_array, window_size=10, overlap=0.75):
    sr = 16000
    window_size_samples = int(window_size * sr)
    overlap_samples = int(window_size_samples * overlap)

    windows = []
    for i in range(0, len(audio_array) - window_size_samples, overlap_samples):
        window = audio_array[i:i + window_size_samples]
        windows.append(window)

    return windows


In [73]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000*10, truncation=True)
    return inputs

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)

    # Calculate precision, recall, and f1 score
    precision = precision_score(y_true=eval_pred.label_ids, y_pred=predictions, average='weighted')
    recall = recall_score(y_true=eval_pred.label_ids, y_pred=predictions, average='weighted')
    f1 = f1_score(y_true=eval_pred.label_ids, y_pred=predictions, average='weighted')

    return {
        "precision": precision,
        "recall": recall,
        "f1_score": f1}

### Training


In [ ]:
from datasets import load_dataset
from datasets import Audio
from datasets import DatasetDict, Dataset
from transformers import AutoFeatureExtractor
import evaluate
import numpy as np
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score,confusion_matrix,precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
from collections import defaultdict
from transformers import EarlyStoppingCallback


#read data
dataset = load_dataset("audiofolder", data_dir="/content/drive/MyDrive/deceptive-16khz",split='train')
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
print(dataset)

# apply windowing to the dataset
windowed_dataset = defaultdict(list)
for example in dataset:
    audio_array = example["audio"]["array"]
    windows = window_audio(audio_array)

    for window in windows:
        windowed_dataset["audio"].append({"array": window})
        windowed_dataset["label"].append(example["label"])

dataset = Dataset.from_dict(windowed_dataset)
print(dataset)
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/hubert-base-ls960")


num_labels = 2
kf = KFold(n_splits=5, shuffle=True, random_state=42)
all_f1_scores = []
all_confusion_matrices = []
for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    print(f"\n----- Fold {fold + 1} -----")

    # Create datasets for this fold
    train_dataset = dataset.select(train_idx)
    test_dataset = dataset.select(test_idx)

    # Preprocess the datasets
    encoded_train_dataset = train_dataset.map(preprocess_function, remove_columns="audio", batched=True)
    encoded_test_dataset = test_dataset.map(preprocess_function, remove_columns="audio", batched=True)

    # Model initialization
    num_labels = 2
    model = AutoModelForAudioClassification.from_pretrained(
        "facebook/hubert-base-ls960", num_labels=num_labels
    )

    training_args = TrainingArguments(
        output_dir= "hubert_deception-1",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=8,
        num_train_epochs=10,
        warmup_ratio=0.1,
        logging_steps=10,
        load_best_model_at_end=True,
        push_to_hub=False,
      )

    feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/hubert-large-ls960-ft")

    trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=encoded_train_dataset,
      eval_dataset=encoded_test_dataset,
      tokenizer=feature_extractor,
      compute_metrics=compute_metrics,
      callbacks=[EarlyStoppingCallback(3, 0.0)]
  )

    trainer.train()

    eval_results = trainer.evaluate()

    # Print F1 score for this fold
    print(f"Fold {fold + 1} - F1 Score: {eval_results['eval_f1_score']}")
        # Append F1 score and confusion matrix to lists
    all_f1_scores.append(eval_results['eval_f1_score'])

# Calculate mean F1 score
mean_f1_score = np.mean(all_f1_scores)
print(f"\nMean F1 Score across all folds: {mean_f1_score}")



Resolving data files:   0%|          | 0/117 [00:00<?, ?it/s]

Dataset({
    features: ['audio', 'label'],
    num_rows: 115
})
Dataset({
    features: ['audio', 'label'],
    num_rows: 333
})

----- Fold 1 -----


Map:   0%|          | 0/266 [00:00<?, ? examples/s]

Map:   0%|          | 0/67 [00:00<?, ? examples/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.weight', 'classifier.bias', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'projector.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Map:   0%|          | 0/267 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.weight', 'classifier.bias', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'projector.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1 Score
0,No log,0.686231,0.683777,0.651515,0.591056
2,0.678200,0.573075,0.725541,0.727273,0.719968
4,0.507000,0.455773,0.755860,0.757576,0.753792
6,0.456700,0.305077,0.921212,0.909091,0.906656
8,0.348400,0.275331,0.932851,0.924242,0.922659
9,0.379100,0.271080,0.932851,0.924242,0.922659


Fold 5 - F1 Score: 0.9226590964973065

Mean F1 Score across all folds: 0.8872739586643499


In [ ]:
from datasets import load_dataset, Audio

dataset = load_dataset("audiofolder", data_dir="/content/drive/MyDrive/deceptive-16khz",split = "train")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

from transformers import pipeline

classifier = pipeline("audio-classification", model="/content/hubert_deception-1/checkpoint-80")
for file in dataset :
  audio = file["audio"]["path"]
  print(classifier(audio))

Resolving data files:   0%|          | 0/117 [00:00<?, ?it/s]

In [ ]:
from transformers import pipeline

classifier = pipeline("audio-classification", model="/content/hubert_deception-1/checkpoint-80")
for file in dataset :
  audio = file["audio"]["path"]
  print(classifier(audio))

[{'score': 0.6854604482650757, 'label': 'LABEL_1'}, {'score': 0.3145395517349243, 'label': 'LABEL_0'}]
[{'score': 0.9022054672241211, 'label': 'LABEL_1'}, {'score': 0.09779457747936249, 'label': 'LABEL_0'}]
[{'score': 0.7441938519477844, 'label': 'LABEL_0'}, {'score': 0.25580617785453796, 'label': 'LABEL_1'}]
[{'score': 0.9032216668128967, 'label': 'LABEL_1'}, {'score': 0.09677833318710327, 'label': 'LABEL_0'}]
[{'score': 0.8836219310760498, 'label': 'LABEL_1'}, {'score': 0.11637815088033676, 'label': 'LABEL_0'}]
[{'score': 0.8940549492835999, 'label': 'LABEL_1'}, {'score': 0.10594508796930313, 'label': 'LABEL_0'}]
[{'score': 0.8764867782592773, 'label': 'LABEL_0'}, {'score': 0.12351320683956146, 'label': 'LABEL_1'}]
[{'score': 0.8802664279937744, 'label': 'LABEL_0'}, {'score': 0.11973357945680618, 'label': 'LABEL_1'}]
[{'score': 0.8819144368171692, 'label': 'LABEL_0'}, {'score': 0.11808554828166962, 'label': 'LABEL_1'}]
[{'score': 0.8738248944282532, 'label': 'LABEL_0'}, {'score': 0.1

# Train Model on Bag of Lies Dataset

### Generate Metadata

In [66]:
import pandas as pd
import re
import os
import csv

annot = pd.read_csv('/content/drive/MyDrive/bag-of-lies/BagOfLies/Annotations.csv')
annot[['video','truth']]
annot['video'] = annot['video'].str.replace("./Finalised/","")
annot['video'] = annot['video'].str.replace("/","_")
annot['video'] = annot['video'].str.replace("_video.mp4","")

def rearrange_string(input_str):
    parts = input_str.split('_')
    rearranged_str = f"{parts[2]}_{parts[3]}_{parts[0]}_{parts[1]}"
    return rearranged_str

for i in range(len(annot['video'])):
  annot['video'][i] = rearrange_string(annot['video'][i])


directory = "/content/drive/MyDrive/bag-of-lies/BagOfLies/Audio"
data = []
for filename in os.listdir(directory):
    if filename.endswith(".wav"):
        user_run = filename.replace(".wav", "")
        label = annot.copy().loc[annot['video'] == user_run]['truth'].reset_index()['truth'][0]
        if label == 1:
          label = 0
        else:
          label = 1
        data.append((filename, label))

csv_file_path = "/content/drive/MyDrive/bag-of-lies/BagOfLies/metadata.csv"


with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["file_name", "label"])
    writer.writerows(data)

print(f"Metadata file created at {csv_file_path}")



<ipython-input-66-68335c1d60e1>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  annot['video'] = annot['video'].str.replace("./Finalised/","")
<ipython-input-66-68335c1d60e1>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  annot['video'] = annot['video'].str.replace("_video.mp4","")
<ipython-input-66-68335c1d60e1>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annot['video'][i] = rearrange_string(annot['video'][i])


Metadata file created at /content/drive/MyDrive/bag-of-lies/BagOfLies/metadata.csv


### Training

In [70]:
from datasets import load_dataset
from datasets import Audio
from datasets import DatasetDict, Dataset
from transformers import AutoFeatureExtractor
import evaluate
import numpy as np
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score,confusion_matrix,precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
from collections import defaultdict
from transformers import EarlyStoppingCallback

In [ ]:
#read data
dataset = load_dataset("audiofolder", data_dir="/content/drive/MyDrive/bag-of-lies/BagOfLies/Audio",split='train')
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# apply windowing to the dataset
windowed_dataset = defaultdict(list)
for example in dataset:
    audio_array = example["audio"]["array"]
    windows = window_audio(audio_array)

    for window in windows:
        windowed_dataset["audio"].append({"array": window})
        windowed_dataset["label"].append(example["label"])

dataset = Dataset.from_dict(windowed_dataset)

########################## training ##############################################

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/hubert-base-ls960")

num_labels = 2
kf = KFold(n_splits=5, shuffle=True, random_state=42)

all_f1_scores = []
all_confusion_matrices = []
for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    print(f"\n----- Fold {fold + 1} -----")

    train_dataset = dataset.select(train_idx)
    test_dataset = dataset.select(test_idx)

    encoded_train_dataset = train_dataset.map(preprocess_function, remove_columns="audio", batched=True)
    encoded_test_dataset = test_dataset.map(preprocess_function, remove_columns="audio", batched=True)

    num_labels = 2
    model = AutoModelForAudioClassification.from_pretrained(
        "facebook/hubert-base-ls960", num_labels=num_labels
    )

    training_args = TrainingArguments(
        output_dir= "hubert_deception-1",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=8,
        num_train_epochs=10,
        warmup_ratio=0.1,
        logging_steps=10,
        load_best_model_at_end=True,
        push_to_hub=False,
      )

    feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/hubert-large-ls960-ft")

    trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=encoded_train_dataset,
      eval_dataset=encoded_test_dataset,
      tokenizer=feature_extractor,
      compute_metrics=compute_metrics,
      callbacks=[EarlyStoppingCallback(3, 0.0)]
  )

    trainer.train()

    eval_results = trainer.evaluate()

    # Print F1 score for this fold
    print(f"Fold {fold + 1} - F1 Score: {eval_results['eval_f1_score']}")
        # Append F1 score and confusion matrix to lists
    all_f1_scores.append(eval_results['eval_f1_score'])

# Calculate mean F1 score
mean_f1_score = np.mean(all_f1_scores)
print(f"\nMean F1 Score across all folds: {mean_f1_score}")


Resolving data files:   0%|          | 0/326 [00:00<?, ?it/s]


----- Fold 1 -----


Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['classifier.bias', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'projector.bias', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Fold 4 - F1 Score: 0.5662067643958992

----- Fold 5 -----


Map:   0%|          | 0/241 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['classifier.bias', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'projector.bias', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1 Score
0,No log,0.704043,0.788679,0.466667,0.396475
1,0.697000,0.697123,0.797000,0.516667,0.472348
2,0.677400,0.693770,0.797000,0.516667,0.472348


In [13]:
"""
import pydub
def convert_video_to_wav(video_file, output_file):
    audio = pydub.AudioSegment.from_file(video_file, format="mp4")
    audio.export(output_file, format="wav")
    if os.path.exists(video_file):
      os.remove(video_file)
"""

"""
import os
def get_video_files(root_folder):
    video_files = []
    for user_folder in os.listdir(root_folder):
        user_path = os.path.join(root_folder, user_folder)
        for run_folder in os.listdir(user_path):
            run_path = os.path.join(user_path, run_folder)
            for filename in os.listdir(run_path):
                if filename.endswith(".mp4") :
                    video_file = os.path.join(run_path, filename)
                    video_files.append(video_file)
    return video_files
"""

In [27]:
"""
for i, video_file in enumerate(video_files):
    user_folder = os.path.basename(os.path.dirname(video_file))
    run_number = os.path.basename(os.path.dirname(os.path.dirname(video_file)))
    filename, extension = os.path.splitext(os.path.basename(video_file))

    output_file = os.path.join("/content/drive/MyDrive/bag-of-lies/BagOfLies/Audio", f"{user_folder}_{run_number}.wav")
    convert_video_to_wav(video_file, output_file)

    print(f"Converted {video_file} to {output_file}")
"""

'\nfor i, video_file in enumerate(video_files):\n    user_folder = os.path.basename(os.path.dirname(video_file))\n    run_number = os.path.basename(os.path.dirname(os.path.dirname(video_file)))\n    filename, extension = os.path.splitext(os.path.basename(video_file))\n\n    output_file = os.path.join("/content/drive/MyDrive/bag-of-lies/BagOfLies/Audio", f"{user_folder}_{run_number}.wav")\n    convert_video_to_wav(video_file, output_file)\n\n    print(f"Converted {video_file} to {output_file}")\n'